In [4]:
import pandas as pd
import plotly
import plotly.express as px
plotly.io.templates.default = 'plotly_white'
from shapely.geometry import LineString, Point

In [5]:
import pandas as pd
all_flights_df = pd.read_excel("powerlinedata/M30 All flights COMBINED Per sec.xlsx")

In [6]:
line_posses = pd.read_csv("powerlinedata/Power Line Heights_Lat_Long.csv")

In [7]:
import geopy.distance
def distance3d(pos1, pos2):
    coords_1 = (pos1[0], pos1[1])
    coords_2 = (pos2[0], pos2[1])
    dist2d = geopy.distance.geodesic(coords_1, coords_2).feet

    alt_diff = abs(pos1[2] - pos2[2])
    dist3d = ((alt_diff * alt_diff) + (dist2d * dist2d))
    return dist3d


In [17]:
from tqdm import tqdm

for aFlight in tqdm(all_flights_df['Flight'].unique()):
    trip_nos = all_flights_df[all_flights_df['Flight'] == 1]['Trip'].unique()

    for trip_no in trip_nos:
        trip_df = all_flights_df[(all_flights_df['Flight'] == aFlight) & (all_flights_df['Trip'] == trip_no)]
        trip_df = trip_df.sort_values(by='Time(cst)')
        trip_df['lat_shift'] = trip_df['latitude'].shift()
        trip_df['lon_shift'] = trip_df['longitude'].shift()
        trip_df['alt_shift'] = trip_df['height_above_takeoff_ft'].shift()
        trip_df = trip_df.dropna(subset=['alt_shift'])
        distance_travelled = []
        for _, a_row in trip_df.iterrows():
            current_lat = a_row['latitude']
            current_lon = a_row['longitude']
            current_alt = a_row['height_above_takeoff_ft']

            prev_lat = a_row['lat_shift']
            prev_lon = a_row['lon_shift']
            prev_alt = a_row['alt_shift']

            distance_3d = distance3d((current_lat, current_lon, current_alt), (prev_lat, prev_lon, prev_alt))
            distance_travelled.append(distance_3d)
        trip_df['dist_consecutive'] = distance_travelled
        trip_df.to_csv(f"./data/flight_{aFlight}_Trip_{trip_no}_with_p2pdist.csv")


100%|██████████| 5/5 [00:01<00:00,  4.94it/s]


In [54]:
total_distance_meters = [] # !!!!!!!!!!!! METERS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
line1 = (47.860236, -97.069306)
line2 = (47.860236, -97.067591)
line_height = 40 / 3.281 # METERS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


from shapely.geometry import LineString, Point
from math import radians, cos, sqrt
def calculate_point_on_line(line1, line2, pointx, pointy):
    line = LineString([line1, line2])
    p = Point(pointx, pointy)
    closest_p =  line.interpolate(line.project(p))
    closest_point = (closest_p.x, closest_p.y)

    R = 6371
    x = (radians(closest_p.y) - radians(pointy)) * cos(0.5 * (radians(closest_p.x) + radians(pointx)))
    y = radians(closest_p.x) - radians(pointx)
    d = R * sqrt(x*x + y*y)
    return d * 1000




In [55]:
all_flights_df = all_flights_df.dropna()
all_flights_df = all_flights_df[all_flights_df != '9\xa0mph']

In [56]:
all_flights_df[all_flights_df != '9\xa0mph']

,Flight,Trip,time,date,Time(cst),latitude,longitude,height_above_takeoff_ft,height_sonar,satellites,...,Temperature,dew_Point,Humidity,Wind angle,wind_speed,wind_gust,Line_Voltage_rating,AC,H_Field_mG,E_Field_V/m
0,1,1,0,2023-01-19 16:39:39,2023-01-19 11:39:39,46.623139,-96.865910,0.000000,0.000000,20,...,22.0,19.756,91.25,0.0,14.256,0,250,0,0.2,82.1
1,1,1,1,2023-01-19 16:39:40,2023-01-19 11:39:40,46.623139,-96.865910,0.000000,0.000000,20,...,22.0,19.756,91.25,0.0,14.256,0,250,0,0.4,81.9
2,1,1,2,2023-01-19 16:39:41,2023-01-19 11:39:41,46.623139,-96.865910,0.000000,0.000000,20,...,22.0,19.756,91.25,0.0,14.256,0,250,0,0.3,81.5
3,1,1,3,2023-01-19 16:39:42,2023-01-19 11:39:42,46.623138,-96.865910,0.656168,0.656168,20,...,22.0,19.756,91.25,0.0,14.256,0,250,0,0.3,81.3
4,1,1,4,2023-01-19 16:39:43,2023-01-19 11:39:43,46.623137,-96.865910,8.202100,8.202100,20,...,22.0,19.756,91.25,0.0,14.256,0,250,0,0.3,82.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4433,5,2,733500,2023-01-18 18:22:37,2023-01-18 12:22:37,48.787053,-95.419562,2.296588,0.984252,23,...,18.0,12.000,79.00,67.5,9,0,500,1,3.9,15.5
4434,5,2,734500,2023-01-18 18:22:38,2023-01-18 12:22:38,48.787054,-95.419562,1.640420,0.328084,23,...,18.0,12.000,79.00,67.5,9,0,500,1,4.6,0.0
4435,5,2,735500,2023-01-18 18:22:39,2023-01-18 12:22:39,48.787054,-95.419562,0.984252,0.000000,23,...,18.0,12.000,79.00,67.5,9,0,500,1,3.6,0.0
4436,5,2,736600,2023-01-18 18:22:40,2023-01-18 12:22:40,48.787054,-95.419562,0.984252,0.000000,23,...,18.0,12.000,79.00,67.5,9,0,500,1,2.9,3.5


In [57]:
import plotly.express as px
from plotly import graph_objects as go
from tqdm import tqdm

total_distance_meters = [] # !!!!!!!!!!!! METERS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# line1 = (47.90568735435933,  -97.11450460041047)
# line2 = (47.9059701437393, -97.1191226016499)
# line_height =50 / 3.281 # METERS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# flight_number = 4

for flight in tqdm(all_flights_df['Flight'].unique()):
    flight_df = all_flights_df[all_flights_df['Flight'] == flight]
    flight_df['altitude_meters'] = flight_df['height_above_takeoff_ft'] / 3.281
    voltage_rating = all_flights_df[all_flights_df['Flight'] == flight]['Line_Voltage_rating'].unique()[0]
    # print(line_posses[line_posses['transmission'] == voltage_rating]['ht'])
    print(voltage_rating)
    line_height = line_posses[line_posses['transmission'] == voltage_rating]['ht'].iloc[0] / 3.281
    start_lat = line_posses[line_posses['transmission'] == voltage_rating]['startlat'].iloc[0]
    start_lon = line_posses[line_posses['transmission'] == voltage_rating]['startlon'].iloc[0]
    end_lat = line_posses[line_posses['transmission'] == voltage_rating]['endlat'].iloc[0]
    end_lon = line_posses[line_posses['transmission'] == voltage_rating]['endlon'].iloc[0]

    flight_dists = []
    for _, aRow in flight_df.iterrows():
        altitude = aRow["altitude_meters"]
        lat = aRow["latitude"]
        lon = aRow["longitude"]
        twoDdist = calculate_point_on_line(line1=(start_lat, start_lon), line2=(end_lat, end_lon), pointx=lat, pointy=lon)
        alt_diff = abs(altitude - line_height)
        total3dDist = sqrt((twoDdist ** 2) + (alt_diff ** 2))
        total_distance_meters.append(total3dDist)
        flight_dists.append(total3dDist)

    f = go.Figure()
    fig = px.line_3d(x=[start_lon, end_lon],
                     y=[start_lat, end_lat],
                     z=[line_height * 3.281, line_height * 3.281] ,
                     color_discrete_sequence=['black', 'black'],
                     hover_name=["line", "line"])

    drone_posses = px.scatter_3d(x=flight_df['longitude'],
                                 y=flight_df['latitude'],
                                 z=flight_df['altitude_meters'].apply(lambda x: x * 3.281),
                                 color=flight_dists)
    f.add_traces(list(drone_posses.select_traces()))
    f.add_traces(list(fig.select_traces()))
    f.update_layout(title=f"Flight No. {flight}, Voltage {voltage_rating}, M300 Line and distance from lin ein meters")
    f.write_html(f"Flight{flight}_LineDistM300.html")

all_flights_df['wire_distance_meters'] = total_distance_meters

# all_flights_df.to_csv("allflightsAddedWiredist")

  0%|          | 0/5 [00:00<?, ?it/s]/tmp/ipykernel_14238/2204984412.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



250


 20%|██        | 1/5 [00:00<00:01,  3.83it/s]/tmp/ipykernel_14238/2204984412.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

 40%|████      | 2/5 [00:00<00:00,  5.57it/s]/tmp/ipykernel_14238/2204984412.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



69
230


 60%|██████    | 3/5 [00:00<00:00,  5.37it/s]/tmp/ipykernel_14238/2204984412.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

 80%|████████  | 4/5 [00:00<00:00,  5.60it/s]/tmp/ipykernel_14238/2204984412.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



345
500


100%|██████████| 5/5 [00:00<00:00,  5.01it/s]


In [58]:
import plotly

In [66]:
all_flights_df = all_flights_df[all_flights_df != '8\xa0mph']

In [67]:
all_flights_df['battery_power'] = all_flights_df['voltage'] * all_flights_df['current']

In [72]:
all_flights_df.columns

Index(['Flight', 'Trip', 'time', 'date', 'Time(cst)', 'latitude', 'longitude',
       'height_above_takeoff_ft', 'height_sonar', 'satellites', 'gpslevel',
       'voltage', ' xSpeed', ' ySpeed', ' zSpeed', ' pitch', ' roll',
       'rc_elevator', 'rc_aileron', 'rc_throttle', 'rc_rudder',
       'battery_percent', 'current', 'battery_temperature', 'flycStateRaw',
       'Payload', 'Temperature', 'dew_Point', 'Humidity', 'Wind angle',
       'wind_speed', 'wind_gust', 'Line_Voltage_rating', 'AC', 'H_Field_mG',
       'E_Field_V/m', 'wire_distance_meters', 'battery_power'],
      dtype='object')

In [74]:
all_flights_df[[i for i in all_flights_df.columns if i not in ['Flight', 'Trip', 'time', 'date', 'Time(cst)', 'latitude', 'longitude', "wind_speed", "wind_gust"]]].corr()

,height_above_takeoff_ft,height_sonar,satellites,gpslevel,voltage,xSpeed,ySpeed,zSpeed,pitch,roll,...,Temperature,dew_Point,Humidity,Wind angle,Line_Voltage_rating,AC,H_Field_mG,E_Field_V/m,wire_distance_meters,battery_power
height_above_takeoff_ft,1.000000,0.402983,-0.592210,NaN,0.063073,0.007691,-0.016272,-0.002185,-0.329171,0.008508,...,-0.742564,-0.754354,0.686717,-0.550360,0.607015,-0.097824,-0.372811,-0.387238,0.040282,-0.082058
height_sonar,0.402983,1.000000,-0.378376,NaN,0.291211,-0.145313,0.083840,-0.038586,-0.257486,-0.068265,...,-0.594117,-0.592373,0.573909,-0.614907,0.094044,-0.531411,-0.405732,-0.492793,-0.148907,-0.072724
satellites,-0.592210,-0.378376,1.000000,NaN,-0.193765,0.007827,-0.010069,-0.021451,0.296684,0.059172,...,0.661334,0.629986,-0.752388,0.799558,-0.088115,0.633646,0.499162,0.495631,-0.029247,0.227818
gpslevel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
voltage,0.063073,0.291211,-0.193765,NaN,1.000000,-0.109996,0.273130,-0.157315,0.047461,-0.341850,...,-0.070307,-0.075119,0.063823,-0.100147,-0.079871,-0.127238,-0.153383,-0.143910,-0.112630,0.115288
xSpeed,0.007691,-0.145313,0.007827,NaN,-0.109996,1.000000,-0.416335,-0.015752,-0.045481,0.226281,...,-0.006441,-0.006269,0.006526,-0.005430,0.003589,-0.002337,0.125695,0.106574,-0.069338,0.005902
ySpeed,-0.016272,0.083840,-0.010069,NaN,0.273130,-0.416335,1.000000,-0.115342,0.163846,-0.734628,...,0.003694,0.004510,-0.000526,0.002924,-0.003666,0.000427,-0.062115,-0.054564,0.004324,0.015775
zSpeed,-0.002185,-0.038586,-0.021451,NaN,-0.157315,-0.015752,-0.115342,1.000000,0.046160,0.049587,...,0.001006,0.000968,-0.001340,0.000816,0.000099,-0.000031,0.020411,0.017926,0.000053,-0.231824
pitch,-0.329171,-0.257486,0.296684,NaN,0.047461,-0.045481,0.163846,0.046160,1.000000,-0.275956,...,0.278158,0.271436,-0.293850,0.297584,-0.090496,0.216050,0.230473,0.232913,-0.000444,0.064718
roll,0.008508,-0.068265,0.059172,NaN,-0.341850,0.226281,-0.734628,0.049587,-0.275956,1.000000,...,0.000049,0.004106,0.007516,-0.004562,0.010492,-0.010644,0.066776,0.062196,0.014934,-0.020874


In [29]:
for j in all_flights_df.columns:

    all_flights_df[[i for i in all_flights_df.columns if i not in [j]]].corr()

ValueError: could not convert string to float: '9\xa0mph'

In [1]:
print(999)

999


In [44]:
flight_d = all_flights_df[all_flights_df['Line_Voltage_rating'] != 500]
flight_d[['E_Field_V/m', 'H_Field_mG', 'wire_distance_meters', 'battery_power']].corr()

,E_Field_V/m,H_Field_mG,wire_distance_meters,battery_power
E_Field_V/m,1.000000,0.884521,-0.496955,0.409814
H_Field_mG,0.884521,1.000000,-0.424403,0.373457
wire_distance_meters,-0.496955,-0.424403,1.000000,-0.179681
battery_power,0.409814,0.373457,-0.179681,1.000000


In [25]:
flight_d = all_flights_df[all_flights_df['Line_Voltage_rating'] == 230]
flight_d[['E_Field_V/m', 'H_Field_mG', 'wire_distance_meters', 'battery_power']].corr()

,E_Field_V/m,H_Field_mG,wire_distance_meters,battery_power
E_Field_V/m,1.000000,0.843642,-0.905219,0.296728
H_Field_mG,0.843642,1.000000,-0.722899,0.242363
wire_distance_meters,-0.905219,-0.722899,1.000000,-0.378635
battery_power,0.296728,0.242363,-0.378635,1.000000


In [26]:
flight_d = all_flights_df[all_flights_df['Line_Voltage_rating'] == 345]
flight_d[['E_Field_V/m', 'H_Field_mG', 'wire_distance_meters', 'battery_power']].corr()

,E_Field_V/m,H_Field_mG,wire_distance_meters,battery_power
E_Field_V/m,1.000000,0.655833,-0.585409,0.199620
H_Field_mG,0.655833,1.000000,-0.435271,0.155176
wire_distance_meters,-0.585409,-0.435271,1.000000,-0.247142
battery_power,0.199620,0.155176,-0.247142,1.000000


In [27]:
flight_d = all_flights_df[all_flights_df['Line_Voltage_rating'] == 500]
flight_d[['E_Field_V/m', 'H_Field_mG', 'wire_distance_meters', 'battery_power']].corr()

,E_Field_V/m,H_Field_mG,wire_distance_meters,battery_power
E_Field_V/m,1.000000,0.725820,-0.273917,-0.103276
H_Field_mG,0.725820,1.000000,-0.505452,-0.100570
wire_distance_meters,-0.273917,-0.505452,1.000000,0.035494
battery_power,-0.103276,-0.100570,0.035494,1.000000


In [29]:
flight_d = all_flights_df[all_flights_df['Line_Voltage_rating'] == 69]
flight_d[['E_Field_V/m', 'H_Field_mG', 'wire_distance_meters', 'battery_power']].corr()

,E_Field_V/m,H_Field_mG,wire_distance_meters,battery_power
E_Field_V/m,1.000000,-0.531807,0.089927,0.012615
H_Field_mG,-0.531807,1.000000,0.018554,-0.062967
wire_distance_meters,0.089927,0.018554,1.000000,-0.185542
battery_power,0.012615,-0.062967,-0.185542,1.000000


In [31]:
flight_d = all_flights_df[all_flights_df['Line_Voltage_rating'].isin( [500, 230, 345])]
flight_d[['E_Field_V/m', 'H_Field_mG', 'wire_distance_meters', 'battery_power']].corr()

,E_Field_V/m,H_Field_mG,wire_distance_meters,battery_power
E_Field_V/m,1.000000,0.834519,-0.650368,0.283739
H_Field_mG,0.834519,1.000000,-0.557951,0.277888
wire_distance_meters,-0.650368,-0.557951,1.000000,-0.216581
battery_power,0.283739,0.277888,-0.216581,1.000000


In [30]:
flight_d = all_flights_df[all_flights_df['Line_Voltage_rating'] == 250]
flight_d[['E_Field_V/m', 'H_Field_mG', 'wire_distance_meters', 'battery_power']].corr()

,E_Field_V/m,H_Field_mG,wire_distance_meters,battery_power
E_Field_V/m,1.000000,0.796122,-0.554201,-0.063176
H_Field_mG,0.796122,1.000000,-0.653665,-0.084164
wire_distance_meters,-0.554201,-0.653665,1.000000,0.013345
battery_power,-0.063176,-0.084164,0.013345,1.000000


In [57]:
bin_label = []
for i in all_flights_df['wire_distance_meters']:
    bin_label.append(str(((int(i/5) * 5) +5)))

In [58]:
all_flights_df['bin'] = bin_label

In [59]:
voltage_color_mapper = {"69": "#2500F5",
                        "230": "#08FFDA",
                        "250": "#98E805",
                        "345": "#FF9F08",
                        "500": "#F50750"
                        }

In [60]:
[voltage_color_mapper[str(Avoltage_rating)]] * len(voltage_Df)

['#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F50750',
 '#F

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

In [80]:



f = go.Figure()
a_bar = px.box(x=all_flights_df['bin'], y = all_flights_df['E_Field_V/m'], color=all_flights_df['Line_Voltage_rating'], )
f.add_traces(list(a_bar.select_traces()))
title = "E filed by distance(Meters), color: Line voltage"
f.update_layout(boxmode='group', title=title)
f.update_xaxes(categoryorder='array', categoryarray= [str(i) for i in range(5, 500, 5)])
f.write_html(title + ".html")
f.show()

In [81]:
f = go.Figure()
a_bar = px.box(x=all_flights_df['bin'], y = all_flights_df['H_Field_mG'], color=all_flights_df['Line_Voltage_rating'], )
f.add_traces(list(a_bar.select_traces()))
title = "H filed by distance(Meters), color: Line voltage"
f.update_layout(boxmode='group', title=title)
f.update_xaxes(categoryorder='array', categoryarray= [str(i) for i in range(5, 500, 5)])
f.write_html(title + ".html")
f.show()